In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.models import mlp_builder as bare_model_builder
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels

In [2]:
PROJECT_NAME = 'mlp'

OVERWRITE = True

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 200
BATCH_SIZE = 8096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all[(2, 11)].shape

((22412, 24), (22412,))

In [4]:
X_train, X_val, y_train, y_val = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train = np.concatenate(list(y_train.values()))
X_val = np.concatenate(list(X_val.values()))
y_val = np.concatenate(list(y_val.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train = X_train[train_idx], y_train[train_idx]
X_val, y_val = X_val[val_idx], y_val[val_idx]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((85651, 24), (85651,), (21416, 24), (21416,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                300       
                                                                 
 dense_1 (Dense)             (None, 18)                234       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 553
Trainable params: 553
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_callbacks = [callbacks.EarlyStopping(patience=30, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=5)]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 18s]
val_loss: 8.833267211914062

Best val_loss So Far: 8.601903915405273
Total elapsed time: 00h 16m 06s


# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 5, 'units_mult': 8, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 3121
========== Model 1 ==========
{'n_hidden_layers': 2, 'units_mult': 16, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': True}
Number of parameters: 3697
========== Model 2 ==========
{'n_hidden_layers': 7, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 61153
========== Model 3 ==========
{'n_hidden_layers': 8, 'units_mult': 16, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': False, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 19249
========== Model 4 ==========
{'n_hidden_layers': 6, 'units_m

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 5, 'units_mult': 8, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 3121
Got score: 8.4485 (8.4404, 8.4567)
Got score: 8.5587 (8.5377, 8.5797)
Got score: 8.5632 (8.5239, 8.6024)
Got score: 8.5297 (8.4962, 8.5633)
Got score: 8.3527 (8.3615, 8.3438)


{'n_hidden_layers': 2, 'units_mult': 16, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': True}
Number of parameters: 3697
Got score: 8.5644 (8.5993, 8.5295)
Got score: 8.7237 (8.7619, 8.6856)
Got score: 8.6727 (8.6575, 8.6878)
Got score: 8.7276 (8.5966, 8.8586)
Got score: 8.5713 (8.5970, 8.5456)


{'n_hidden_layers': 7, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 61153
Got score: 9.0755 (8.8425, 9.3084)
Got score: 8.8962 (8.8845, 8.9079)
Got score: 8.7627 (8.7556, 8.7699)
Got score: 8.9462 (8.7259, 9.1666)
Got score: 8.7137 (8.8231, 8.6044)


{'n_hidden_layers': 8, 'units_mult': 16, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': False, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 19249
Got score: 9.2770 (9.6636, 8.8904)
Got score: 14.5057 (9.9795, 19.0319)
Got score: 9.8766 (9.8975, 9.8556)
Got score: 10.3060 (9.6632, 10.9487)
Got score: 9.0822 (9.0761, 9.0883)


{'n_hidden_layers': 6, 'units_mult': 2, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 607
Got score: 8.9954 (9.0562, 8.9346)
Got score: 9.1078 (9.1450, 9.0706)
Got score: 9.3299 (8.9131, 9.7467)
Got score: 8.9261 (8.9412, 8.9111)
Got score: 9.3579 (10.0738, 8.6420)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.49,0.08,3121
1,8.65,0.07,3697
2,8.88,0.13,61153
3,10.61,2.00,19249
4,9.14,0.17,607
